# Django ORM - Praca z danymi (komendy DML i DQL)

**Definicje modelu**

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from form_app5.models import Task

## R - Read (CRUD) - DQL (Data Query Language) cd

### Składanie warunków (operator AND)

Wypisywanie kolejnych warunków po przecinku, jako kolejnych parametrów funkcji filter powoduje składanie tych warunków (koniunkcje/iloczyn logiczną) - operator AND.

Znajdż wszystkie wpisy w tabeli Task, które zaczynają się na Pr **i** kończą na anie.

In [3]:
# metoda I
task = Task.objects.filter(name__startswith="Pr").filter(name__endswith="anie")

# metoda II
tasks = Task.objects.filter(name__startswith="Pr", name__endswith="anie")
print(tasks)

<QuerySet [<Task: Programowanie>, <Task: Pranie>]>


### Sortowanie - metoda order_by (klauzula ORDER BY)

Klauzula ORDER BY implementowana jest w Django przez metodę order_by klasy QuerySet. Metoda order_by jako parametr przyjmuje nazwę kolumny, po której dane mają zostać posortowane. Może przyjąć więcej niż jeden parametr, wtedy wpisy które mają identyczną wartość w pierwszej kolumnie będą sortowane po drugiej wpisanej kolumnie, itd. Wartości w kolumnach tekstowych sortowane są alfabetycznie, a wartości w kolumnach liczbowych numerycznie.

Posortujemy wpisy w tabeli Task po wartościach w kolumnie name.

In [4]:
tasks = Task.objects.order_by('name')
for task in tasks:
    print(f"{task.id} {task.name}")

10 Dodawanie
3 Gotowanie
1 Mycie
2 Pieczenie
9 Pranie
7 Programowanie
8 Pływanie
4 Smażenie
5 Szukanie
6 Szukanie


In [5]:
# sql?
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" ORDER BY "form_app5_task"."name" ASC


Posortujmy wpisy w tabeli Task po kolumnie name, a wpisy które mają identyczną wartość w kolumnie name po kolumnie id.

In [6]:
tasks = Task.objects.order_by('name', 'id')
for task in tasks:
    print(f"{task.id} {task.name}")

10 Dodawanie
3 Gotowanie
1 Mycie
2 Pieczenie
9 Pranie
7 Programowanie
8 Pływanie
4 Smażenie
5 Szukanie
6 Szukanie


#### Sortowanie odwrotne - Metoda I (metoda reverse)

Sortować w kolejności odwrotnej możemy z użyciem metody reverse()

In [7]:
tasks = Task.objects.order_by('name').reverse()
for task in tasks:
    print(f"{task.id} {task.name}")

5 Szukanie
6 Szukanie
4 Smażenie
8 Pływanie
7 Programowanie
9 Pranie
2 Pieczenie
1 Mycie
3 Gotowanie
10 Dodawanie


In [8]:
print(tasks.query)

SELECT "form_app5_task"."id", "form_app5_task"."name" FROM "form_app5_task" ORDER BY "form_app5_task"."name" DESC


#### Sortowanie odwrotne - Metoda II (nazwa kolumny)

lub poprzez wstawienie przed nazwą kolumny minusa.

In [9]:
tasks = Task.objects.order_by('-name')
for task in tasks:
    print(f"{task.id} {task.name}")

5 Szukanie
6 Szukanie
4 Smażenie
8 Pływanie
7 Programowanie
9 Pranie
2 Pieczenie
1 Mycie
3 Gotowanie
10 Dodawanie


### Funkcja agregujące

Funkcje agregujące w sql to operatory, które wykonują na wskazanych danych proste statystyki takie jak średnia, wartość maksymalna, suma czy liczba wpisów (SUM, AVG, MIN, MAX, COUNT). W Django funkcje agregujące realizowane są przez metodę aggregate. Metoda aggregate jako parametr przyjmuje wywołanie odpowiedniej klasy z modułu django.db.models (Sum, Avg, Min, Max, Count, ...).

Znajdź sumę indeksów wszystkich wpisów tabeli Task. 

In [10]:
from django.db.models import Sum

a_sum = Task.objects.aggregate(Sum('id'))
print(a_sum)

{'id__sum': 55}


Funkcja agregująca zwraca słownik z kluczem 
< nazwa_kolumny_wzdluz_ktorej_zachodzi_agregacja >_ _<nazwa_funkcji_agregującej> oraz wartością będąca wyliczoną statystyką.

Znajdź średnią wartość indeksu wpisów tabeli Task o wartości w kolumnie name "Szukanie"

In [11]:
from django.db.models import Avg

avg = Task.objects.filter(name="Szukanie").aggregate(Avg('id'))
print(avg)

{'id__avg': 5.5}


## U - UPDATE (CRUD) - DML (Data Manipulation Language)

### Klauzula UPDATE

In [13]:
# Metoda I - metoda update mandżera modelu (i QuerySet-a)
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE ZMIENIĆ WSZYSTKICH wpisów 
# w tabeli.

tasks = Task.objects.filter(name__endswith="enie").update(name="GGotowanie")
print(tasks)

2


Widzimy, że metoda update nie zwraca nam obiektu klasy QuerySet tylko liczbę zmodyfikowanych wpisów.

In [12]:
# Metoda II - bezpośrednia modyfikacja wartości atrybutu instancji modelu

task = Task.objects.get(name="Dodawanie")
task.name = "Odejmowanie"
task.save()

## D - DELETE (CRUD) - DML (DATA Manipulation Language)

### Klauzula DELETE

In [15]:
# Metoda I - metoda delete menadżera modelu (i QuerySet-a)
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE USUNĄĆ WSZYSTKICH wpisów 
# w tabeli.

task = Task.objects.filter(name="Pływanie").delete()
print(task)

(1, {'form_app5.Task': 1})


Podobnie jak metoda update, metoda delete nie zwraca nam obiektu klasy Queryset tylko informacje o liczbie usuniętych wpisów (tym razem w postaci tupli, której pierwszy element to całkowita liczba usuniętych wpisów, a drugi element to słownik z kluczami będącymi nazwami modelu i wartościami będącymi liczbą usuniętych w danym modelu wpisów).

In [14]:
# Metoda II - metoda delete instancji modelu

task = Task.objects.get(name="Odejmowanie")
task.delete()

(1, {'form_app5.Task': 1})

Metoda delete instancji modelu zwraca nam identyczną odpowiedź co metoda delete menadżera modelu (i Queryset-a)